In [ ]:
import sys
sys.path.insert(1,'/wsu/home/gy/gy40/gy4065/hm.jetscapeml.source')

import numpy as np

In [ ]:
print('Loading/Installing Package => Begin\n\n')

import jet_ml_dataset_builder.jet_ml_dataset_builder_utilities as util

print('\n########################################################################')
print('Checking the running platforms\n')

from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import set_directory_paths
# Call the function and retrieve the dataset_directory_path and simulation_directory_path
dataset_directory_path, simulation_directory_path = set_directory_paths()

# Access the dataset_directory_path and simulation_directory_path
print("Dataset Directory Path:", dataset_directory_path)
print("Simulation Directory Path:", simulation_directory_path)
print('########################################################################\n')


print('\nLoading/Installing Package => End\n\n')


In [ ]:
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import  parse_parameters

# Call the function and retrieve the tokenized parameters
tokenized_arguments, tokenized_values = parse_parameters()

# Access the tokenized arguments and values
print("Tokenized Arguments:")
for argument in tokenized_arguments:
    print(argument)

print("\nTokenized Values:")
for argument, value in tokenized_values.items():
    print(f"{argument}: {value}")

y_class_label_items=['MMAT','MLBT']
alpha_s_items=[0.2 ,0.3 ,0.4]
q0_items=[1.5 ,2.0 ,2.5]

print("y_class_label_items:",y_class_label_items)
print("alpha_s_items:",alpha_s_items)
print("q0_items:",q0_items)

# Loading the dataset


In [ ]:
print("Building required params for the loading the dataset file")

class_labels_str = '_'.join(y_class_label_items)
alpha_s_items_str='_'.join(map(str, alpha_s_items))
q0_items_str='_'.join(map(str, q0_items))
total_size=9*1200000
# for shuffled_y_processed
# dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{total_size}_split_train_datasets/train_split_0.pkl"
# for shuffled
dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{1000}_shuffled.pkl"
# dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{10000}_shuffled.pkl"
# dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{100000}_shuffled.pkl"
# dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{1000000}_shuffled.pkl"
# dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{total_size}_shuffled.pkl"

dataset_file_name=simulation_directory_path+dataset_file_name
print("dataset_file_name:",dataset_file_name)

- The first column of `dataset_y` is extracted (`dataset_y_binary`) for binary classification.
- The dataset is split into training and testing sets using `train_test_split`.

implementing k-fold for the model

In [ ]:
# loading libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [ ]:
# loading dataset by size and getting just the first column

from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import load_dataset
# Function to load datasets of different sizes
def get_dataset(size):
    dataset_file_name = f"jet_ml_benchmark_config_01_to_09_alpha_{alpha_s_items_str}_q0_{q0_items_str}_{class_labels_str}_size_{size}_shuffled.pkl"
    
    dataset_file_name=simulation_directory_path+dataset_file_name
    print("dataset_file_name:",dataset_file_name)
    
    dataset=load_dataset(dataset_file_name,has_test=False)
    (dataset_x, dataset_y) = dataset
    # Extract the first column for binary classification
    dataset_y = dataset_y[:, 0]
    print("dataset.x:",type(dataset_x), dataset_x.size, dataset_x.shape)
    print("dataset.y:",type(dataset_y), dataset_y.size,dataset_y.shape)
    return dataset_x, dataset_y

In [ ]:
# defining dataset sizes and classifiers

# Sizes of datasets
# dataset_sizes = [1000]
# dataset_sizes = [1000, 10000]
dataset_sizes = [1000, 10000, 100000, 1000000]


# Classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'LinearSVC': LinearSVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=42)
}

In [ ]:
# Function to train and evaluate classifiers 
def train_and_evaluate_classifier(model, x_train, y_train, x_test, y_test):
    # Assuming x and y are defined
    # x should be a 2D array (e.g., (1000, 32*32))
    # y should be a 2D array with three columns (e.g., (1000, 3))

    # Flatten the 32x32 images to 1D arrays for LogisticRegression, DecisionTreeClassifier, LinearSVM, KNN, RandomForests
    x_train_flatten = x_train.reshape(x_train.shape[0], -1)
    x_test_flatten = x_test.reshape(x_test.shape[0], -1)
    model.fit(x_train_flatten, y_train)
    y_pred = model.predict(x_test_flatten)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return accuracy, cm

In [ ]:
# Function to train and evaluate classifiers using k-fold cross-validation and giving confusion matrix and accuracy as results
def train_and_evaluate_classifier_kfold(model, x, y, k_fold=5):
    x_flatten = x.reshape(x.shape[0], -1)
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    
    scores = []
    confusion_matrices = []

    for train_index, test_index in kf.split(x_flatten):
        x_train, x_test = x_flatten[train_index], x_flatten[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        scores.append(accuracy)
        confusion_matrices.append(cm)

    return scores, confusion_matrices

In [ ]:
# Results storage
results_kfold = []
results_kfold_errorbar=[]
# Loop through different dataset sizes
for size in dataset_sizes:
    # Generate dataset
    x, y = get_dataset(size)
    
    # Loop through classifiers
    for clf_name, clf in classifiers.items():
        # Evaluate classifier using k-fold cross-validation
        fold_scores, fold_conf_matrices = train_and_evaluate_classifier_kfold(clf, x, y)
        
        # Store results for each fold
        for fold_num, (score, cm) in enumerate(zip(fold_scores, fold_conf_matrices), start=1):
            results_kfold.append({
                'Dataset Size': size,
                'Classifier': clf_name,
                'Fold Number': fold_num,
                'Accuracy': score,
                'Confusion Matrix': cm
            })
        # Calculate mean and standard deviation of accuracy scores
        mean_accuracy = np.mean(fold_scores)
        std_accuracy = np.std(fold_scores)
        
        # Store results
        results_kfold_errorbar.append({
            'Dataset_Size': size,
            'Classifier': clf_name,
            'Mean_Accuracy': mean_accuracy,
            'Std_Accuracy': std_accuracy
        })
# Create a DataFrame from k-fold results
df_results_kfold = pd.DataFrame(results_kfold)
# Save the DataFrame to a text file
df_results_kfold.to_csv('binary_classification_results_kfold.txt', index=False, sep='\t')
# Display results in a table
print(df_results_kfold)

# Create a DataFrame from k-fold results
df_results_kfold_errorbar = pd.DataFrame(results_kfold_errorbar)
# Save the DataFrame to a text file
df_results_kfold_errorbar.to_csv('binary_classification_results_kfold_errorbar.txt', index=False, sep='\t')
# Display results in a table
print(df_results_kfold_errorbar)

In [ ]:
# Define the module labels
module_labels = ['MMATTER', 'MLBT']

In [ ]:
# Save confusion matrices for each fold
for index, row in df_results_kfold.iterrows():
    clf_name = row['Classifier']
    dataset_size = row['Dataset Size']
    fold_num = row['Fold Number']
    cm = row['Confusion Matrix']

    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap='Oranges') # plt.cm.Blues

    # Annotate each cell with the value
    for i in range(len(module_labels)):
        for j in range(len(module_labels)):
            plt.text(j, i, str(cm[i, j]), ha='center', va='center', color='black')

    plt.title(f'Confusion Matrix - {clf_name} - {dataset_size} samples - Fold {fold_num}')
    plt.colorbar()
    # Set tick labels
    plt.xticks(np.arange(len(module_labels)), module_labels)
    plt.yticks(np.arange(len(module_labels)), module_labels)

    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    # Remove tick marks
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    plt.savefig(f'confusion_matrix_{clf_name}_{dataset_size}_fold_{fold_num}.png', dpi=300)
    plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Load the DataFrame from the saved file
#df_results = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/binary_classification_results_kfold_errorbar.txt", sep='\t')
df_results= df_results_kfold_errorbar
print(df_results)
# Set a seaborn style (optional)
sns.set(style="whitegrid")


# Define a dictionary to map classifiers to markers
marker_dict = {
    'Logistic Regression': 'o',
    'Decision Tree': 's',
    'LinearSVC': '^',
    'KNN': 'v',
    'Random Forest': 'D'
}
plt.figure(figsize=(10, 6))

for clf_name, group in df_results.groupby('Classifier'):
    plt.errorbar(
        group['Dataset_Size'],
        group['Mean_Accuracy'],
        yerr=group['Std_Accuracy'],
        label=clf_name,
        marker=marker_dict.get(clf_name, 'o'),  # Use 'o' as default marker if not found in the dictionary
        capsize=5
    )

plt.xscale('log')  # Set x-axis to logarithmic scale
plt.xlabel('Dataset Size (log scale)')
plt.ylabel('Mean Accuracy')
plt.title('Binary Classification Accuracy with Error Bars for Different Dataset Sizes')
plt.legend()
# plt.grid(True)
# Save the plot with high resolution (300 dpi)
plt.savefig('binary_classification_accuracy_errorbar_plot.png', dpi=300)
plt.show()